
### Objective: 

In this assignment, implement the Node2Vec algorithm, a random-walk-based GNN, to learn node embeddings. Train a classifier using the learned embeddings to predict node labels.

### Dataset: 

Cora dataset: The dataset consists of 2,708 nodes (scientific publications) with 5,429 edges (citations between publications). Each node has a feature vector of size 1,433, and there are 7 classes (research topics).
Skeleton Code:

In [1]:
import numpy as np
print(np.__version__)

1.26.4


In [2]:
#pip install numpy==1.24.4 scipy==1.10.1

In [6]:
!pip install node2vec

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
Using cached scipy-1.13.1-cp311-cp311-win_amd64.whl (46.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyfume 0.3.4 requires numpy==1.24.4, but you have numpy 1.26.4 which is incompatible.
pyfume 0.3.4 requires scipy==1.10.1, but you have scipy 1.13.1 which is incompatible.


In [4]:
#import node2vec
#print(node2vec.__version__)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from node2vec import Node2Vec


# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

def test(model):
    model.eval()
    with torch.no_grad():
        out = model(dataset.data)
        pred = out.argmax(dim=1)
        flag_is_correct = pred[dataset.data.test_mask] == dataset.data.y[dataset.data.test_mask]
        acc = int(flag_is_correct.sum()) / int(dataset.data.test_mask.sum())
        return acc, pred[dataset.data.test_mask], dataset.data.y[dataset.data.test_mask]
# Training loop

acc_lst = []

for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')
    acc_lst.append(test(classifier)[0])

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

C:\Users\siddu\AppData\Local\Temp\ipykernel_6848\2527165497.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
C:\Users\siddu\anaconda3\Lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Epoch 0, Loss: 1.9437246322631836


TypeError: linear(): argument 'input' (position 1) must be Tensor, not Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from node2vec import Node2Vec
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Convert embeddings to a single tensor
embedding_tensor = torch.tensor(np.array([embeddings[str(i)] for i in range(data.num_nodes)]), dtype=torch.float)

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Modified test function to use embeddings
def test(model):
    model.eval()
    with torch.no_grad():
        out = model(embedding_tensor)
        pred = out.argmax(dim=1)
        flag_is_correct = pred[data.test_mask] == data.y[data.test_mask]
        acc = int(flag_is_correct.sum()) / int(data.test_mask.sum())
        return acc

# Training loop
acc_lst = []

for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(embedding_tensor)
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')
    
    acc_lst.append(test(classifier))

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Epoch 0, Loss: 2.0141940116882324
Epoch 10, Loss: 1.2898032665252686
Epoch 20, Loss: 0.9287654161453247
Epoch 30, Loss: 0.7653117775917053
Epoch 40, Loss: 0.6859518885612488
Epoch 50, Loss: 0.6399396657943726
Epoch 60, Loss: 0.6095027327537537
Epoch 70, Loss: 0.5872741937637329
Epoch 80, Loss: 0.5700252652168274
Epoch 90, Loss: 0.5560986995697021
Training complete!


In [2]:
acc_lst

[0.229,
 0.366,
 0.419,
 0.467,
 0.494,
 0.519,
 0.54,
 0.569,
 0.593,
 0.622,
 0.636,
 0.65,
 0.669,
 0.683,
 0.698,
 0.71,
 0.723,
 0.733,
 0.745,
 0.753,
 0.758,
 0.762,
 0.765,
 0.768,
 0.771,
 0.773,
 0.779,
 0.781,
 0.783,
 0.79,
 0.791,
 0.792,
 0.792,
 0.791,
 0.792,
 0.795,
 0.797,
 0.799,
 0.799,
 0.796,
 0.797,
 0.798,
 0.802,
 0.806,
 0.806,
 0.805,
 0.805,
 0.805,
 0.806,
 0.807,
 0.806,
 0.807,
 0.808,
 0.808,
 0.809,
 0.81,
 0.81,
 0.812,
 0.813,
 0.813,
 0.815,
 0.815,
 0.816,
 0.818,
 0.817,
 0.819,
 0.82,
 0.821,
 0.821,
 0.821,
 0.821,
 0.821,
 0.821,
 0.821,
 0.821,
 0.822,
 0.822,
 0.825,
 0.826,
 0.826,
 0.826,
 0.825,
 0.825,
 0.827,
 0.826,
 0.827,
 0.827,
 0.827,
 0.827,
 0.828,
 0.828,
 0.828,
 0.828,
 0.828,
 0.828,
 0.829,
 0.828,
 0.829,
 0.829,
 0.829]

## Explanation:
Node2Vec generates node embeddings by simulating random walks on the graph. These walks capture structural properties of nodes.
The generated embeddings are then used to train a classifier for predicting node labels.
The Cora dataset is a benchmark graph where nodes are papers and edges are citations.

## Questions (1 point each):
1. What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?
2. What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?
4. What would happen if we used directed edges instead of undirected edges for the random walks?
5. What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?
6. What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?
8. What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?



### Extra credit: 
1. What would happen if we increased the window size (window) for the skip-gram model? How would it affect the embedding quality?

## No points, just for you to think about
7. What would happen if we removed self-loops from the graph before training Node2Vec?

9. What would happen if we applied normalization to the node embeddings before feeding them to the classifier?

## Answers ##
1) If we increase the number of walks, we would get more embeddings from node2vec, so the model has more data to train from. This should theoretically increase the accuracy. However, if num_walks is too high, the model will memorize the entire graph and overfit the data.
2) Reducing the walk length would give us the same number of embeddings, but the sizes of these embeddings would be smaller, i.e, we would have less features. This means the model would learn more local  information and less global. This could work well in tasks like recommender systems, and higher walk lenghts would work for overall structure detections tasks such as graph classification.
3) Directed edges would have better accuracy in tasks which care about directionality, such as emails, where a person A sending an email to B doesn't mean B is sending the same email to A. Undirected egdes would be better for tasks which don't care about directionality, such as social networks where A being B's friend is the same as B being A's friend.
4) If we use more features, the accuracy should be higher, unless all of them contain unnecessary information and don't help the model learn any more than it already did.
5) I think the classifier performance would decrease with an increase in the number of classes. To get a better accuracy, you would have to increase the number of training iterations for the model to learn about the entire data.
6) This case is similar to increasing the walk length. Increasing the size of embeddings means that the model has more to learn about each node. If the model has a high node dievrsity, this would improve the accuracy, but since we're passing in more data to the model, the training would take longer time.


## EC ##
1) I believe that the result would be same as that of increasing walk length. Since the window size decides how many neighbors encountered in each walk we look at, a higher window size means the model would be learning more global information, and lower window size would help it learn more local information. 